In [1]:
import json
import copy

from pprint import pprint

from next_to_left_func import next_to_left_function
from top_or_bottom_func import top_or_bottom_function
from corner_intersection_func import corner_intersection_function
from int_16_9_and_17_6 import intersection_16_9_and_17_6_function
from int_17_9_and_16_6 import intersection_17_9_and_16_6_function
from left_func import left_function
from helper import *

In [2]:
top_intersections = {f"intersection_{i}_9" for i in range(10, 31) if i not in {16, 17}}
bottom_intersections = {f"intersection_{i}_6" for i in range(10, 31) if i not in {16, 17}}
left_intersections = {"intersection_10_7", "intersection_10_8"}
special_intersections = {"intersection_16_9", "intersection_17_9", "intersection_16_6", "intersection_17_6"}

next_to_top = {f"intersection_{i}_10" for i in range(11, 30) if i != 16}
next_to_left = {"intersection_9_7", "intersection_9_8"}
next_to_bottom = {f"intersection_{i}_5" for i in range(11, 30) if i != 17}

int_types = {}

for column in range(1, 31):
    for row in range(1, 12):
        
        intersection = f"intersection_{column}_{row}"
        
        #if intersection in all_special_intersections:
        #    continue
                
        #### TOP ###
        if row >= 9:
            
            int_types[intersection] = {"x": -400, "y": 120 + 170 * (row - 7)}
            
        elif row >= 7:
            if column <= 10:
                int_types[intersection] = {"x": -400, "y": 120 + 170 * (row - 7)}
                
        ### BOTTOM ###
        else:
            int_types[intersection] = {"x": -400, "y": -300 + 50 * (row - 1)}

In [3]:
path = "../../Simulation_Results/Manhattan/"
roadnet = path + "roadnet_original.json"
grid = path + "roadnet_temp.json"


with open(roadnet, 'r') as f:
    roadnet_data = json.load(f)
    
with open(grid, 'r') as f:
    grid_data = json.load(f)
    

#### NOT SAME SHIFT EVERYWHERE
modify_intersection = list(int_types.keys())
    
set_to_modify_roads = set()

### MODIFY INTERSECTIONS
for int_index, intersection in enumerate(grid_data["intersections"]):
    if intersection["id"] in modify_intersection:
        
        intersection_id = intersection["id"]
        intersection_id_list = intersection_id.split("_")
        
        intersection_index_roadnet = find_intersection_index(roadnet_data, intersection_id)
        
        left_IN_lane = f"road_{int(intersection_id_list[1]) - 1}_{int(intersection_id_list[2])}_0"
        left_OUT_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2])}_2"
        
        bottom_IN_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2]) - 1}_1"
        bottom_OUT_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2])}_3"
        
        right_IN_lane = f"road_{int(intersection_id_list[1]) + 1}_{int(intersection_id_list[2])}_2"
        right_OUT_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2])}_0"
        
        top_IN_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2]) + 1}_3"
        top_OUT_lane = f"road_{int(intersection_id_list[1])}_{int(intersection_id_list[2])}_1"
        
        # add lanes such that we add another lane to it later when we modify all roads
        set_to_modify_roads.update({left_IN_lane, right_IN_lane})
        
        
        if intersection_id in next_to_left:
            next_to_left_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, bottom_IN_lane, top_IN_lane, intersection_index_roadnet, intersection_id, intersection)
            # add lanes from intersection left and right to the modify list
            set_to_modify_roads.update(left_OUT_lane)
            continue
        
        else:
            # add lanes from intersection left and right to the modify list
            set_to_modify_roads.update({left_OUT_lane, right_OUT_lane})
            
            
        if intersection_id in {"intersection_10_6", "intersection_10_9", "intersection_30_6", "intersection_30_9"}:
            corner_intersection_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, top_IN_lane, bottom_IN_lane, top_intersections, intersection_index_roadnet, intersection_id, intersection)
            continue
            
        if intersection_id in {"intersection_17_9", "intersection_16_6"}:
            intersection_17_9_and_16_6_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, bottom_IN_lane, top_IN_lane, intersection_index_roadnet, intersection_id, intersection)
            continue
        
        if intersection_id in {"intersection_16_9", "intersection_17_6"}:
            intersection_16_9_and_17_6_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, bottom_IN_lane, top_IN_lane, intersection_index_roadnet, intersection_id, intersection)
            continue
            
        if intersection_id in top_intersections.union(bottom_intersections):
            top_or_bottom_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, top_IN_lane, bottom_IN_lane, top_intersections, intersection_index_roadnet, intersection_id, intersection)
            continue
            
        if intersection_id in left_intersections:
            left_function(roadnet_data, int_types, left_IN_lane, right_IN_lane, top_IN_lane, bottom_IN_lane, intersection_index_roadnet, intersection_id, intersection)
            continue
          
        # Add another lane
        for roadLink_index, roadLink in enumerate(intersection["roadLinks"]):
            
            # add another lane starting from left or right
            if roadLink["startRoad"] in [left_IN_lane, right_IN_lane]:
                for laneLink_index, laneLink in enumerate(roadLink["laneLinks"]):
                    
                    # add the Connection from the old straight lane to the new lane on the other side (avenue -> avenue)
                    if laneLink["startLaneIndex"] == 1 and laneLink["endLaneIndex"] == 3:
                        copy_laneLink = copy.deepcopy(laneLink)
                        shift_points(int_types, intersection_id, copy_laneLink)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].insert(laneLink_index, copy_laneLink)
                        
                    # add all Connections from the new lane (avenue -> avenue)
                    if laneLink["startLaneIndex"] == 2:
                        copy_laneLink = copy.deepcopy(laneLink)
                        shift_points(int_types, intersection_id, copy_laneLink)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].insert(laneLink_index, copy_laneLink)
                        
                    # change the right turn (index and points) of the original file (avenue -> street)
                    if laneLink["startLaneIndex"] == 3 and laneLink["endLaneIndex"] <= 2:
                        
                        # next_to_top only has 2 downstream lanes
                        if intersection_id in next_to_top and roadLink["endRoad"] == bottom_OUT_lane and laneLink["endLaneIndex"] == 2:
                            continue
                          
                        # next_to_bottom only has 2 downstream lanes
                        if intersection_id in next_to_bottom and roadLink["endRoad"] == top_OUT_lane and laneLink["endLaneIndex"] == 2:
                            continue
                            
                        copy_laneLink = copy.deepcopy(laneLink)
                        shift_points(int_types, intersection_id, copy_laneLink)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].pop(laneLink_index)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].insert(laneLink_index, copy_laneLink)
          
                    
            # add another lane starting from bottom or top
            if roadLink["startRoad"] in [bottom_IN_lane, top_IN_lane]:
                for laneLink_index, laneLink in enumerate(roadLink["laneLinks"]):
                    # = 3 --> right turn (street -> avenue)
                    if laneLink["startLaneIndex"] == 3 and laneLink["endLaneIndex"] == 3:
                        copy_laneLink = copy.deepcopy(laneLink)
                        shift_points(int_types, intersection_id, copy_laneLink)
                        # change the startLaneIndex
                        copy_laneLink["startLaneIndex"] = 2
                        
                        # change the points
                        points_x = roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"][0]["points"]
                        points_y = copy_laneLink["points"]
                        
                        copy_laneLink["points"] = mix_points(points_x, points_y)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].insert(laneLink_index, copy_laneLink)
                        
                    # = 0 --> left turn (street -> avenue)
                    if laneLink["startLaneIndex"] == 0 and laneLink["endLaneIndex"] == 3:
                        copy_laneLink = copy.deepcopy(laneLink)
                        shift_points(int_types, intersection_id, copy_laneLink)
                        roadnet_data["intersections"][intersection_index_roadnet]["roadLinks"][roadLink_index]["laneLinks"].insert(laneLink_index, copy_laneLink)
              
            
            



### MODIFY ROADS
new_lane = {"width": 4, "maxSpeed": 16.67}
for road in roadnet_data["roads"]:
    if road["id"] in set_to_modify_roads:
        road["lanes"].append(new_lane)
                        

                        
                        
### SAVE NEW FILE
# Save the modified data as a new JSON file
new_json_file = path + "roadnet.json"
with open(new_json_file, "w") as f:
    json.dump(roadnet_data, f, indent=4)

print(f"Modified data saved as {new_json_file}")

Modified data saved as ../../Simulation_Results/Manhattan/roadnet.json


In [4]:
test(path, new_json_file)